# Analysis of WT and RPM basal organoids +/- RPM basal-organoid-derived allograft tumours
## Ireland et al 2024 BioRxiv
## Updated 05.2025
### Related to Fig. 2d and Extended Data Fig. 4a-d (Final Fig. 1i and Ext. Data Fig. 4a-d)

### First, analysis of wildtype basal organoids and RPM organoids "CellTagged pre-Cre" and "CellTagged post-Cre" for Extended Data Fig. 4a-b

In [ ]:
#Import other relevant packages
import numpy as np
import pandas as pd
from matplotlib import rcParams
import os
import scanpy as sc

import matplotlib as mpl
import matplotlib.pyplot as plt

#For nice color schemes
import cmocean

#For barplots
import seaborn as sns

#Import scVI
import scvi
from scvi.model.utils import mde

scvi.settings.verbosity = 40

#Import scVI
import scvi
from scvi.model.utils import mde

scvi.settings.verbosity = 40

In [ ]:
sc.settings.verbosity = 3             # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_header()
sc.settings.set_figure_params(dpi=80, facecolor='white')

In [ ]:
# Read 
os.chdir('/work/asi16')

## 1. Read in all in vitro RPM organoid data, original and new

In [ ]:
# Read in RPM TBOs "CellTagged pre-Cre" 10X matrices aligned to CellTag/GFP.CDS genome + Luciferase/Venus
rpm_org_cre=sc.read_10x_mtx('042024_custom_count_RPMTBO_CMV_Pool1/RPM/count/sample_filtered_feature_bc_matrix/', var_names='gene_symbols', cache=True)

In [ ]:
# Read in RPM TBOs "CellTagged post-Cre" 10X matrices aligned to CellTag/GFP.CDS genome + Luciferase/Venus
rpm_org_cre_new=sc.read_10x_mtx('042225_RPM_TBO_CellTagPostCre/042325_RPMTBO_InVitro_CTpostCre/outs/filtered_feature_bc_matrix/', var_names='gene_symbols', cache=False)

In [ ]:
# Add appropriate metadata to samples
rpm_org_cre.obs['Genotype'] = 'RPM'
rpm_org_cre.obs['Model'] = 'Organoid'
rpm_org_cre.obs['Cre'] = 'CT_preCre'
rpm_org_cre.obs['UnID'] = 'RPM_Org_Cre'
rpm_org_cre.obs['Batch'] = 'Org_CTpreCre'

rpm_org_cre_new.obs['Genotype'] = 'RPM'
rpm_org_cre_new.obs['Model'] = 'Organoid'
rpm_org_cre_new.obs['Cre'] = 'CT_postCre'
rpm_org_cre_new.obs['UnID'] = 'RPM_Org_Cre'
rpm_org_cre_new.obs['Batch'] = 'Org_CTpostCre'


In [ ]:
rpm_org_cre_new

## Also read in wildtype basal organoid data

In [ ]:
orgs_nocre=sc.read_10x_mtx('TBO_Pool_NoCre_NotCellPlexed/042024_custom_count_TBOpoolNoCellPlex/filtered_feature_bc_matrix/', var_names='gene_symbols', cache=True)

In [ ]:
# Add appropriate metadata
orgs_nocre.obs['Genotype'] = 'WT'
orgs_nocre.obs['Model'] = 'Organoid'
orgs_nocre.obs['Cre'] = 'No_Cre'
orgs_nocre.obs['UnID'] = 'WT_Org_NoCre'
orgs_nocre.obs['Batch'] = 'Org_No_Cre'

In [ ]:
#Subsample bc >20k cells, keep 10k to be more comparable with RPM transformed organoi samples
orgs_nocre_subset=sc.pp.subsample(orgs_nocre, n_obs=10000, copy=True)

In [ ]:
# Add metadata IDs
orgs_nocre_subset.obs['Genotype'] = 'WT'
orgs_nocre_subset.obs['Model'] = 'Organoid'
orgs_nocre_subset.obs['Cre'] = 'No_Cre'
orgs_nocre_subset.obs['UnID'] = 'WT_Org_NoCre'
orgs_nocre_subset.obs['Batch'] = 'Org_No_Cre'

## 2. Concatenate WT and RPM transformed organoid datasets

In [ ]:
# Add the No Cre RPM to Plus Cre organoid samples and check it
#Concatenate datasets
orgs_all= rpm_org_cre.concatenate([rpm_org_cre_new,orgs_nocre], index_unique=None, join="outer")

## 3. Perform QC

In [ ]:
orgs_all

In [ ]:
#QC filtering
orgs_all.var['mito'] = orgs_all.var_names.str.startswith('mt-')  # annotate the group of mitochondrial genes as 'mt'
sc.pp.calculate_qc_metrics(orgs_all, qc_vars=['mito'], percent_top=None, log1p=False, inplace=True)
#QC filtering RPMA RPM only scanpy flow
sc.pp.filter_cells(orgs_all, min_genes=200)
#sc.pp.filter_genes(orgs_all, min_cells=3)

orgs_all.var['mito'] = orgs_all.var_names.str.startswith('mt-')  # annotate the group of mitochondrial genes as 'mt'
sc.pp.calculate_qc_metrics(orgs_all, qc_vars=['mito'], percent_top=None, log1p=False, inplace=True)

In [ ]:
sc.pl.violin(orgs_all, ['n_genes_by_counts', 'total_counts', 'pct_counts_mito'],
             jitter=0.4, multi_panel=True)
sc.pl.scatter(orgs_all, x='total_counts', y='pct_counts_mito')
sc.pl.scatter(orgs_all, x='total_counts', y='n_genes_by_counts')

In [ ]:
#Filter data by slicing anndata object
orgs_all = orgs_all[orgs_all.obs.n_genes_by_counts < 7000, :]
orgs_all = orgs_all[orgs_all.obs.n_genes_by_counts > 1000, :]
orgs_all = orgs_all[orgs_all.obs.total_counts > 2000, :]
orgs_all = orgs_all[orgs_all.obs.pct_counts_mito < 15, :]

In [ ]:
orgs_all.obs.groupby(["UnID"]).apply(len)

In [ ]:
orgs_all.obs.groupby(["Batch"]).apply(len)

## 4. Identify HVGs with Poisson gene selection

In [ ]:
#Prep for HVG and scvi
#log1p the data
orgs_all.obs["log1p_total_counts"] = np.log1p(orgs_all.obs["total_counts"])
#Create layers
orgs_all.layers["counts"] = orgs_all.X.copy()
orgs_all.layers['norm'] = orgs_all.X.copy(); sc.pp.normalize_total(orgs_all, target_sum=1e4, layer="norm")

In [ ]:
#HVG via Scanpy
#Note here that if you run with a batch_key with few cells, will get b'reciprocal condition number error
sc.pp.highly_variable_genes(
    orgs_all,n_top_genes=5000,
    subset=False,
    layer="counts",
    flavor="seurat_v3"
)

In [ ]:
orgs_all.var['mean_'] = np.array(orgs_all.X.mean(0))[0]
orgs_all.var['frac_zero'] = 1 - np.array((orgs_all.X > 0).sum(0))[0] / orgs_all.shape[0]

fig, ax = plt.subplots(figsize=(9,6))
ax.scatter(orgs_all.var.mean_, orgs_all.var.frac_zero, s=1)
ax.set_xscale("log")

In [ ]:
#Calculate Poisson gene selection
df_poisson = scvi.data.poisson_gene_selection(
    orgs_all, n_top_genes=5000, batch_key="Batch", inplace=False
)

df_poisson[df_poisson.highly_variable].sort_values('prob_zero_enrichment_rank')

pd.crosstab(df_poisson.highly_variable, orgs_all.var.highly_variable)

is_hvg = df_poisson.highly_variable

orgs_all.varm['df_poisson']= df_poisson

adata_query = orgs_all[:, is_hvg].copy()
print(adata_query)

## 5. Set up and train model (scvi)

In [ ]:
scvi.model.SCVI.setup_anndata(
    adata_query,
    layer="counts",
    batch_key='Batch',
    continuous_covariate_keys=["pct_counts_mito"]
)

model = scvi.model.SCVI(adata_query)

In [ ]:
#Train and run scvi

#Training parameters
train_kwargs = dict(
    early_stopping=True,
    early_stopping_patience=20,
    enable_model_summary=True,
    enable_progress_bar=True,
    enable_checkpointing=True,
    max_epochs=500
)

#Train and run model
#Be sure GPU is enabled to run this
model.train(**train_kwargs)

In [ ]:
#Fit model to data
#Get latent representation of model to apply to UMAP
latent = model.get_latent_representation()

orgs_all.obsm["X_scVI_1.1"] = latent

## 6. Perform leiden clustering

In [ ]:
#Calculate neighbors using scVI model input
sc.pp.neighbors(orgs_all, use_rep="X_scVI_1.1")
sc.tl.umap(orgs_all, min_dist=0.5)

#Run leiden clustering based on neighbors
sc.tl.leiden(orgs_all, key_added="leiden_scVI_1.1", resolution=0.5)

## 7. Visualize clustering, QC metrics, and cluster markers

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))
sc.pl.umap(orgs_all, color="Cre", cmap="cmo.matter", s=30, ax=ax, vmax="p99.99", frameon=False, save=False)
fig, ax = plt.subplots(figsize=(8, 6))
sc.pl.umap(orgs_all, color="leiden_scVI_1.1", legend_loc="on data", legend_fontsize='large',ax=ax, s=30, frameon=False, save=False)
fig, ax = plt.subplots(figsize=(8, 6))
sc.pl.umap(orgs_all, color="UnID", legend_loc="right margin", ax=ax, s=30, frameon=False, save=False)

#Additional QC bar graphs
orgs_all.obs['cluster'] = orgs_all.obs["leiden_scVI_1.1"].copy()

#Plot Log1p total counts
fig, ax = plt.subplots(figsize=(20,6))
sns.boxenplot(data=orgs_all.obs, x="cluster", y="log1p_total_counts", ax=ax)

#Plot Pct counts mito
fig, ax = plt.subplots(figsize=(20,6))
sns.boxenplot(data=orgs_all.obs, x="cluster", y="pct_counts_mito", ax=ax)


In [ ]:
# Find cluster markers for each leiden cluster to aid filtering
sc.tl.rank_genes_groups(orgs_all, 'leiden_scVI_1.1', method='wilcoxon', layer='norm', use_raw=False)
pd.DataFrame(orgs_all.uns['rank_genes_groups']['names']).head(50)

In [ ]:
orgs_all.write_h5ad("050125_RPM_WT_CTpostCre_CTpreCre_TBOs_adata.h5ad")

In [ ]:
# Exclude 7 as poor quality.
bad_clust=['7']

#Filter out bad clusters
to_keep=(~orgs_all.obs['leiden_scVI_1.1'].isin(bad_clust))

#Copy over to new anndata object
orgs_all2 = orgs_all[to_keep].copy()

### From here, continue iterating through runs of scvi modeling until no clear low quality cell clusters or non-tumor cells are observed.
### Start back up at "set up and train scvi model" and run through subsetting out "bad clusters".
### Each time clusters are removed, model is run again to recluster.

## ITERATION 2
### Final iteration for Ext. Data Fig. 4a-c

In [ ]:
#HVG via Scanpy
#Note here that if you run with a batch_key with few cells, will get b'reciprocal condition number error
sc.pp.highly_variable_genes(
    orgs_all2,n_top_genes=5000,
    subset=False,
    layer="counts",
    flavor="seurat_v3",
    batch_key="Batch"
)

In [ ]:
orgs_all2.var['mean_'] = np.array(orgs_all2.X.mean(0))[0]
orgs_all2.var['frac_zero'] = 1 - np.array((orgs_all2.X > 0).sum(0))[0] / orgs_all2.shape[0]

fig, ax = plt.subplots(figsize=(9,6))
ax.scatter(orgs_all2.var.mean_, orgs_all2.var.frac_zero, s=1)
ax.set_xscale("log")

In [ ]:
#Calculate Poisson gene selection
df_poisson = scvi.data.poisson_gene_selection(
    orgs_all2, n_top_genes=5000, batch_key="Batch", inplace=False
)

df_poisson[df_poisson.highly_variable].sort_values('prob_zero_enrichment_rank')

pd.crosstab(df_poisson.highly_variable, orgs_all2.var.highly_variable)

is_hvg = df_poisson.highly_variable

orgs_all2.varm['df_poisson']= df_poisson

adata_query = orgs_all2[:, is_hvg].copy()
print(adata_query)

In [ ]:
scvi.model.SCVI.setup_anndata(
    adata_query,
    layer="counts",
    batch_key='Batch',
    continuous_covariate_keys=["pct_counts_mito"]
)

model = scvi.model.SCVI(adata_query)

In [ ]:
#Train and run scvi

#Training parameters
train_kwargs = dict(
    early_stopping=True,
    early_stopping_patience=20,
    enable_model_summary=True,
    enable_progress_bar=True,
    enable_checkpointing=True,
    max_epochs=500
)

#Train and run model
#Be sure GPU is enabled to run this
model.train(**train_kwargs)

In [ ]:
#Fit model to data
#Get latent representation of model to apply to UMAP
latent = model.get_latent_representation()

orgs_all2.obsm["X_scVI_1.2"] = latent

In [ ]:
#Calculate neighbors using scVI model input
sc.pp.neighbors(orgs_all2, use_rep="X_scVI_1.2")
sc.tl.umap(orgs_all2, min_dist=1)

#Run leiden clustering based on neighbors
sc.tl.leiden(orgs_all2, key_added="leiden_scVI_1.2", resolution=0.5)

## Visualize UMAP by sample and Leiden cluster (Ext. Data Fig. 4a)

In [ ]:
# Visualize. Extended Data Fig 4a

fig, ax = plt.subplots(figsize=(6, 6))
sc.pl.umap(orgs_all2, color="leiden_scVI_1.2", legend_loc="right margin", legend_fontsize='xx-large',ax=ax, s=60, frameon=False, save=False)
fig, ax = plt.subplots(figsize=(6, 6))
sc.pl.umap(orgs_all2, color="Cre", legend_loc="right margin", ax=ax, s=60, frameon=False, save=False, palette=['purple','orange'])
fig, ax = plt.subplots(figsize=(6, 6))
sc.pl.umap(orgs_all2, color="UnID", legend_loc="right margin", ax=ax, s=30, frameon=False, save=False)

In [ ]:
orgs_all2.obs.groupby(["Batch"]).apply(len)

In [ ]:
#Additional QC bar graphs
orgs_all2.obs['cluster'] = orgs_all2.obs["leiden_scVI_1.2"].copy()

#Plot Log1p total counts
fig, ax = plt.subplots(figsize=(20,6))
sns.boxenplot(data=orgs_all2.obs, x="cluster", y="log1p_total_counts", ax=ax)

#Plot Pct counts mito
fig, ax = plt.subplots(figsize=(20,6))
sns.boxenplot(data=orgs_all2.obs, x="cluster", y="pct_counts_mito", ax=ax)

## Create DotPlot (Ext. Data. Fig. 4b)

In [ ]:
sc.tl.dendrogram(orgs_all2, groupby="leiden_scVI_1.2")

In [ ]:
# Dot plot key cell type markers (Extended Data Fig. 4b)
more_types=[ "Pdpn","Cav1","Cav2","Hopx","Timp3","Sema3f","Serpine1", #AT1
              "Abca3","Muc1","Sftpa1","Sftpb","Sftpd","Scd1", #AT2
              "Scgb1a1","Cyp2f2","Scgb3a2", "Scgb3a1","Lypd2",#Club
              "Muc5ac","Muc5b", "Spdef",# Goblet
              "Tubb4a","Foxa3","Foxj1","Rfx2","Rfx3","Trp73", #Ciliated
              'Krt5', 'Krt17','Krt15','Krt8','Trp63','Sox2','Id1','Epas1','Aqp3',
            'Sfn','Perp','Fxyd3','Sdc1','Gstm2','F3','Adh7', # Basal
              'Ascl1','Bex1','Insm1','Chga','Chgb','Myt1','Sez6','Foxa2','Sox11','Syp',#NE/neuronal
            "Neurod1","Nhlh1","Nhlh2",
            'Atoh1','Ush2a','Lhx3','Rasd2','Pou4f3', # Atoh1/Inner Ear
              'Pou2f3','Trpm5','Ascl2',
              'Lrmp','Gng13','Ltc4s','Alox5ap','Avil','Alox5','Atp2a3', #tuft
              "Cftr",'Foxi1', "Ascl3", 'Stap1','Atp6v1c2','Pparg','Rasd1', #ionocyte
              'Yap1','Wwtr1','Sox2','Cd44','Hes1', # Stem-like
             "Top2a","Mki67", "Ube2c","Aspm",
            'Myc',"fLuc"] # Tumor markers

# Dot plot for Extended Data Fig. 4e
sc.set_figure_params(scanpy=True, fontsize=20)  # controls default font sizes

dotplot = sc.pl.dotplot(
    orgs_all2,
    var_names=more_types,
    groupby='leiden_scVI_1.2',
    use_raw=False,
    layer="norm",
    figsize=[30, 3],
    show=False,
    color_map="cmo.dense",
    var_group_rotation=35,
    smallest_dot=10,
    dendrogram=True
)

# Increase x-axis label size
plt.xticks(fontsize=30)  # adjust size as needed
plt.yticks(fontsize=30)
plt.tight_layout()
plt.show()

In [ ]:
orgs_all2.write_h5ad("050125_RPM_WT_CTpostCre_CTpreCre_TBOs_adata2.h5ad")

In [ ]:
orgs_all2=sc.read_h5ad("050125_RPM_WT_CTpostCre_CTpreCre_TBOs_adata2.h5ad")

In [ ]:
# Find cluster markers for each leiden cluster for supplementary tables
sc.tl.rank_genes_groups(orgs_all2, 'leiden_scVI_1.2', method='wilcoxon', layer='norm', use_raw=False)
pd.DataFrame(orgs_all2.uns['rank_genes_groups']['names']).head(50)

In [ ]:
# Extract the ranked genes (names) for all clusters
marker_genes = pd.DataFrame(orgs_all2.uns['rank_genes_groups']['names'])

# Save to CSV
marker_genes.to_csv('050125_orgs_all2_RPM_WT_CTpostCre_CTpreCre_cluster_marker_genes.csv', index=False)

# Combining WT and RPM organoid samples, as analyzed above, with resulting RPM allograft tumours
### Related to Fig. 2d (Final Fig. 1i) and Extended Data Fig. 4d

## 1. Read in RPM basal-organoid-derived allograft data

In [ ]:
######### Now cluster in with RPM TBO Allograft tumor! ############

In [ ]:
# Read in original "RPM CellTagged pre-Cre" as RPM_allo and new "RPM CellTagged post-Cre" as RPM_allo3 from 10X matrices
RPM_allo=sc.read_10x_mtx('MedGenome_FASTQ_123123/RPM_TBO_Allo/042024_RPM_TBO_Allo_CustomCount/filtered_feature_bc_matrix', var_names='gene_symbols', cache=True)
RPM_allo3=sc.read_10x_mtx('042225_RPM_TBO_CellTagPostCre/042225_RPMTBOAllo_CTpostCre_1/outs/filtered_feature_bc_matrix', var_names='gene_symbols', cache=True)


In [ ]:
# Add metadata identifiers

RPM_allo.obs['Genotype'] = 'RPM'
RPM_allo.obs['Model'] = 'Allograft'
RPM_allo.obs['Cre'] = 'CT_preCre'
RPM_allo.obs['UnID'] = 'RPM_Allo_Original'
RPM_allo.obs['Batch'] = 'RPM_Allo_Original'
RPM_allo.obs['GenoCT'] = 'RPM_Allo_OG_CTpreCre'

RPM_allo3.obs['Genotype'] = 'RPM'
RPM_allo3.obs['Model'] = 'Allograft'
RPM_allo3.obs['Cre'] = 'CT_postCre'
RPM_allo3.obs['UnID'] = 'RPM_Allo_3_New'
RPM_allo3.obs['Batch'] = 'RPM_Allo_3_New'
RPM_allo3.obs['GenoCT'] = 'RPM_Allo_3_CTpostCre'


## 2. Concatenate allograft data with organoid data

In [ ]:
# Add to organoid samples and check it
#Concatenate datasets
adata= orgs_all2.concatenate([RPM_allo, RPM_allo3], index_unique=None, join="outer")

In [ ]:
adata.obs.groupby(["Batch"]).apply(len)

In [ ]:
adata.obs.groupby(["UnID"]).apply(len)

## 3. Perform QC

In [ ]:
#QC filtering
adata.var['mito'] = adata.var_names.str.startswith('mt-')  # annotate the group of mitochondrial genes as 'mt'
sc.pp.calculate_qc_metrics(adata, qc_vars=['mito'], percent_top=None, log1p=False, inplace=True)

In [ ]:
#QC filtering RPMA RPM only scanpy flow
sc.pp.filter_cells(adata, min_genes=200)

adata.var['mito'] = adata.var_names.str.startswith('mt-')  # annotate the group of mitochondrial genes as 'mt'
sc.pp.calculate_qc_metrics(adata, qc_vars=['mito'], percent_top=None, log1p=False, inplace=True)

In [ ]:
sc.pl.violin(adata, ['n_genes_by_counts', 'total_counts', 'pct_counts_mito'],
             jitter=0.4, multi_panel=True)
sc.pl.scatter(adata, x='total_counts', y='pct_counts_mito')
sc.pl.scatter(adata, x='total_counts', y='n_genes_by_counts')

In [ ]:
#Filter data by slicing anndata object
adata = adata[adata.obs.n_genes_by_counts < 9000, :]
adata = adata[adata.obs.n_genes_by_counts > 2000, :]
adata = adata[adata.obs.total_counts > 2000, :]
adata = adata[adata.obs.pct_counts_mito < 15, :]


## 4. Identify HVGs with Poisson gene selection

In [ ]:
#Prep for HVG and scvi
#log1p the data
adata.obs["log1p_total_counts"] = np.log1p(adata.obs["total_counts"])

In [ ]:
#Create layers
adata.layers["counts"] = adata.X.copy()
adata.layers['norm'] = adata.X.copy(); sc.pp.normalize_total(adata, target_sum=1e4, layer="norm")

In [ ]:
#HVG via Scanpy
#Note here that if you run with a batch_key with few cells, will get b'reciprocal condition number error
sc.pp.highly_variable_genes(
    adata,n_top_genes=10000,
    subset=False,
    layer="counts",
    flavor="seurat_v3",
    batch_key="Batch"
)

In [ ]:
adata.var['mean_'] = np.array(adata.X.mean(0))[0]
adata.var['frac_zero'] = 1 - np.array((adata.X > 0).sum(0))[0] / adata.shape[0]

fig, ax = plt.subplots(figsize=(9,6))
ax.scatter(adata.var.mean_, adata.var.frac_zero, s=1)
ax.set_xscale("log")

In [ ]:
#Calculate Poisson gene selection
df_poisson = scvi.data.poisson_gene_selection(
    adata, n_top_genes=5000, batch_key="Batch", inplace=False
)

df_poisson[df_poisson.highly_variable].sort_values('prob_zero_enrichment_rank')

pd.crosstab(df_poisson.highly_variable, adata.var.highly_variable)

is_hvg = df_poisson.highly_variable

adata.varm['df_poisson']= df_poisson

adata_query = adata[:, is_hvg].copy()
print(adata_query)

## 5. Set up and train scVI model

In [ ]:
scvi.model.SCVI.setup_anndata(
    adata_query,
    layer="counts",
    batch_key='Batch',
    continuous_covariate_keys=["pct_counts_mito"]
)

model = scvi.model.SCVI(adata_query)

In [ ]:
#Train and run scvi

#Training parameters
train_kwargs = dict(
    early_stopping=True,
    early_stopping_patience=20,
    enable_model_summary=True,
    enable_progress_bar=True,
    enable_checkpointing=True,
    max_epochs=500
)

#Train and run model
#Be sure GPU is enabled to run this
model.train(**train_kwargs)

In [ ]:
#Fit model to data
#Get latent representation of model to apply to UMAP
latent = model.get_latent_representation()

adata.obsm["X_scVI_1.1"] = latent

## 6. Perform leiden clustering

In [ ]:
#Calculate neighbors using scVI model input
sc.pp.neighbors(adata, use_rep="X_scVI_1.1")
sc.tl.umap(adata, min_dist=0.5)

#Run leiden clustering based on neighbors
sc.tl.leiden(adata, key_added="leiden_scVI_1.1", resolution=1)

## 7. Visualize clustering, QC metrics, and individual genes to determine tumour vs non-tumour clusters

In [ ]:
#QC UMAPs
sc.pl.umap(
    adata,
    color=["n_genes_by_counts", "total_counts", "pct_counts_mito", "log1p_total_counts"],
    cmap="cubehelix_r",
    s=3,
    ncols=2,
)

In [ ]:
fig, ax = plt.subplots(figsize=(6, 6))
sc.pl.umap(adata, color="Genotype", cmap="cmo.matter", s=10, ax=ax, vmax="p99.99", frameon=False, save=False)
fig, ax = plt.subplots(figsize=(6, 6))
sc.pl.umap(adata, color="leiden_scVI_1.1", legend_loc="on data", legend_fontsize='xx-large',ax=ax, s=10, frameon=False, save=False)
fig, ax = plt.subplots(figsize=(6, 6))
sc.pl.umap(adata, color="UnID", legend_loc="right margin", ax=ax, s=10, frameon=False, save=False)

#Additional QC bar graphs
adata.obs['cluster'] = adata.obs["leiden_scVI_1.1"].copy()

#Plot Log1p total counts
fig, ax = plt.subplots(figsize=(20,6))
sns.boxenplot(data=adata.obs, x="cluster", y="log1p_total_counts", ax=ax)

#Plot Pct counts mito
fig, ax = plt.subplots(figsize=(20,6))
sns.boxenplot(data=adata.obs, x="cluster", y="pct_counts_mito", ax=ax)

In [ ]:
#feature plots
more_types=["Col14a1", "Acta2","Myh11","Tagln","Mustn1", #fibroblast
              "Lpl","Lipa","Pparg","Plin2","Ear1","Fabp1","Spp1", #lipofibroblast/osteoblastic
              "Ptprc","Mertk","Marco","Mrc1","Ly75","Adgre1",
            "Itgax","Cd68","Csf1r","Mafb","Msr1","Arg1","Adgre4","Clec4a1", #Macs/Myeloid
              "Cx3cr1","Itgam","Cd14", #Monocytes
              "S100a9","S100a8","Mmp9","Csf3r","Cxcr2","Ly6g", #Neuts
              "Batf3","Xcr1","Clec9a","Ccl17","Ccl22", #DC
              "Cd3d","Cd3g","Cd28","Cd8a","Cd4","Foxp3", # Tcell
              "Gzma","Ncr1","Gzmb", #NK
              "Fcmr","Fcer2a","Pax5","Cd22","Cd79b","Cd79a", #B cells
              "Slamf7", "Prdm1", #Plasma
              "Mcam","Pecam1","Icam2","Cd36","Cd93", "Ascl1","Neurod1","Pou2f3", "fLuc", "GFP.CDS","CellTag.UTR"] #Endothelial

sc.pl.umap(
    adata,
    color=more_types,
    use_raw=False,
    legend_loc= "on data",
    color_map="cmo.dense",
    ncols=4,s=30,
    frameon=False,
    vmax="p99.5",
    layer="norm",
    save=False
)

In [ ]:
#feature plots
more_types=["Ascl1", "Neurod1","Atoh1","Pou2f3","Trp63","fLuc", "GFP.CDS","CellTag.UTR"] #Endothelial

sc.pl.umap(
    adata,
    color=more_types,
    use_raw=False,
    legend_loc= "on data",
    color_map="cmo.dense",
    ncols=4,s=30,
    frameon=False,
    vmax="p99.5",
    layer="norm",
    save=False
)

In [ ]:
# Dot plot key cell type markers
more_types=["Col14a1", "Acta2","Myh11","Tagln","Mustn1", #fibroblast
              "Lpl","Lipa","Pparg","Plin2","Ear1","Spp1", #lipofibroblast/osteoblastic
              "Ptprc","Mertk","Mrc1","Ly75","Adgre1","Itgax","Cd68","Csf1r","Mafb","Msr1","Arg1","Adgre4", #Macs/Myeloid
              "Cx3cr1","Itgam","Cd14", #Monocytes
              "S100a9","S100a8","Mmp9","Csf3r","Cxcr2","Ly6g", #Neuts
              "Batf3","Xcr1","Clec9a","Ccl17","Ccl22", #DC
              "Cd3d","Cd3e","Cd3g","Cd28","Cd8a","Cd4","Foxp3", # Tcell
              "Gzma","Ncr1","Gzmb", #NK
              "Fcmr","Cd19","Fcer2a","Pax5","Cd22","Cd79b","Cd79a", #B cells
              "Slamf7", "Prdm1", #Plasma
              "Mcam","Pecam1","Icam2","Cd36","Cd93", #Endothelial
    "Pdpn","Cav1","Cav2","Hopx","Timp3","Sema3f","Serpine1", #AT1
              "Abca3","Muc1","Sftpa1","Sftpb","Sftpd","Scd1", #AT2
              "Scgb1a1","Cyp2f2","Scgb3a2", "Scgb3a1","Lypd2",#Club
              "Muc5ac","Muc5b", # Goblet
              "Tubb4a","Foxa3","Foxj1","Rfx2","Rfx3","Trp73", #Ciliated
              'Krt5', 'Krt17','Krt15','Trp63','Id1','Icam1','Epas1','Aqp3','Sfn','Perp','Fxyd3','Sdc1','Gstm2','F3','Abi3bp','Adh7', # Basal
              'Bex2','Ascl1','Meis2','Hes6','Hoxb5','Foxa2','Sox4','Rora','Isl1','Id4', 'Neurod1','Neurod4','Nhlh1','Nhlh2',#NE/neuronal
              'Pou2f3','Trpm5','Ascl2','Ehf',
              'Lrmp','Gng13','Ltc4s','Alox5ap','Avil','Alox5','Atp2a3','Plk2', #tuft
              "Cftr","Ascl3", 'Stap1','Atp6v1c2','Pparg','Rasd1','Slc12a2', #ionocyte
              "Gja1","Nkx2-1","Epcam", # Lung lineage
              'Yap1','Wwtr1','Sox2','Cd44','Hes1', # Stem-like
             "Venus","fLuc", "Top2a","Mki67",
            'GFP.CDS','CellTag.UTR'] # Tumor markers

sc.set_figure_params(scanpy=True, fontsize=20) 
sc.pl.dotplot(
    adata,figsize=[28,10],
    var_names=more_types,
    groupby='leiden_scVI_1.1',
    use_raw=False,
    layer="norm",show=False,
    color_map="cmo.dense", var_group_rotation=35,standard_scale='var',
    save=False)

In [ ]:
# Find cluster markers for each leiden cluster to aid filtering
sc.tl.rank_genes_groups(adata, 'leiden_scVI_1.1', method='wilcoxon', layer='norm', use_raw=False)
pd.DataFrame(adata.uns['rank_genes_groups']['names']).head(50)

In [ ]:
# Extract the ranked genes (names) for all clusters
marker_genes = pd.DataFrame(adata.uns['rank_genes_groups']['names'])

# Save to CSV
marker_genes.to_csv('050225_RPM_Orgs_Allo_adata_DEGsbyleiden.csv', index=False)

In [ ]:
adata.write_h5ad("050225_RPM_Orgs_Allo_UpdatedwAlloOGand3_adata.h5ad")

In [ ]:
# Remove low quality or non-tumour clusters
# 24 is fibroblast, 23 is endothelial, 19 is macs/immune

bad_clust=['24','23','19']

#Filter out bad clusters
to_keep=(~adata.obs['leiden_scVI_1.1'].isin(bad_clust))

#Copy over to new anndata object
adata_2 = adata[to_keep].copy()

### From here, continue iterating through runs of scvi modeling until no clear low quality cell clusters or non-tumor cells are observed.
### Start back up at "set up and train scvi model" and run through subsetting out "bad clusters".
### Each time clusters are removed, model is run again to recluster.

## ITERATION 2

In [ ]:
#HVG via Scanpy
#Note here that if you run with a batch_key with few cells, will get b'reciprocal condition number error
sc.pp.highly_variable_genes(
    adata_2,n_top_genes=5000,
    subset=False,
    layer="counts",
    flavor="seurat_v3",
    batch_key="Batch"
)


In [ ]:
adata_2.var['mean_'] = np.array(adata_2.X.mean(0))[0]
adata_2.var['frac_zero'] = 1 - np.array((adata_2.X > 0).sum(0))[0] / adata_2.shape[0]

fig, ax = plt.subplots(figsize=(9,6))
ax.scatter(adata_2.var.mean_, adata_2.var.frac_zero, s=1)
ax.set_xscale("log")

In [ ]:
#Calculate Poisson gene selection
df_poisson = scvi.data.poisson_gene_selection(
    adata_2, n_top_genes=5000, batch_key="Batch", inplace=False
)

df_poisson[df_poisson.highly_variable].sort_values('prob_zero_enrichment_rank')

pd.crosstab(df_poisson.highly_variable, adata_2.var.highly_variable)

is_hvg = df_poisson.highly_variable

adata_2.varm['df_poisson']= df_poisson

adata_query = adata_2[:, is_hvg].copy()
print(adata_query)

In [ ]:
scvi.model.SCVI.setup_anndata(
    adata_query,
    layer="counts",
    batch_key='Batch',
    continuous_covariate_keys=["pct_counts_mito"]
)

model = scvi.model.SCVI(adata_query)

In [ ]:
#Train and run scvi

#Training parameters
train_kwargs = dict(
    early_stopping=True,
    early_stopping_patience=20,
    enable_model_summary=True,
    enable_progress_bar=True,
    enable_checkpointing=True,
    max_epochs=500
)

#Train and run model
#Be sure GPU is enabled to run this
model.train(**train_kwargs)

In [ ]:
#Fit model to data
#Get latent representation of model to apply to UMAP
latent = model.get_latent_representation()

adata_2.obsm["X_scVI_1.2"] = latent

## Perform leiden clustering

In [ ]:
#Calculate neighbors using scVI model input
sc.pp.neighbors(adata_2, use_rep="X_scVI_1.2")
sc.tl.umap(adata_2, min_dist=0.5)

#Run leiden clustering based on neighbors
sc.tl.leiden(adata_2, key_added="leiden_scVI_1.2", resolution=0.5)

### Visualize clustering, QC metrics, and gene expression to determine tumour vs non-tumour clusters

In [ ]:
#QC UMAPs
sc.pl.umap(
    adata_2,
    color=["n_genes_by_counts", "total_counts", "pct_counts_mito", "log1p_total_counts"],
    cmap="cubehelix_r",
    s=3,
    ncols=2,
)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))
sc.pl.umap(adata_2, color="leiden_scVI_1.2", legend_loc="on data", legend_fontsize='large',ax=ax, s=30, frameon=False, save=False)
fig, ax = plt.subplots(figsize=(8, 6))
sc.pl.umap(adata_2, color="UnID", legend_loc="right margin", ax=ax, s=30, frameon=False, save=False)
fig, ax = plt.subplots(figsize=(8, 6))
sc.pl.umap(adata_2, color="Genotype", legend_loc="right margin", ax=ax, s=30, frameon=False, save=False)
fig, ax = plt.subplots(figsize=(8, 6))
sc.pl.umap(adata_2, color="GenoCT", legend_loc="right margin", ax=ax, s=30, frameon=False, save=False)

#Additional QC bar graphs
adata_2.obs['cluster'] = adata_2.obs["leiden_scVI_1.2"].copy()

#Plot Log1p total counts
fig, ax = plt.subplots(figsize=(20,6))
sns.boxenplot(data=adata_2.obs, x="cluster", y="log1p_total_counts", ax=ax)

#Plot Pct counts mito
fig, ax = plt.subplots(figsize=(20,6))
sns.boxenplot(data=adata_2.obs, x="cluster", y="pct_counts_mito", ax=ax)

In [ ]:
# Dot plot key cell type markers
more_types=["Col14a1", "Acta2","Myh11","Tagln","Mustn1", #fibroblast
              "Lpl","Lipa","Pparg","Plin2","Ear1","Spp1", #lipofibroblast/osteoblastic
              "Ptprc","Mertk","Mrc1","Ly75","Adgre1","Itgax","Cd68","Csf1r","Mafb","Msr1","Arg1","Adgre4", #Macs/Myeloid
              "Cx3cr1","Itgam","Cd14", #Monocytes
              "S100a9","S100a8","Mmp9","Csf3r","Cxcr2","Ly6g", #Neuts
              "Batf3","Xcr1","Clec9a","Ccl17","Ccl22", #DC
              "Cd3d","Cd3e","Cd3g","Cd28","Cd8a","Cd4","Foxp3", # Tcell
              "Gzma","Ncr1","Gzmb", #NK
              "Fcmr","Cd19","Fcer2a","Pax5","Cd22","Cd79b","Cd79a", #B cells
              "Slamf7", "Prdm1", #Plasma
              "Mcam","Pecam1","Icam2","Cd36","Cd93", #Endothelial
    "Pdpn","Cav1","Cav2","Hopx","Timp3","Sema3f","Serpine1", #AT1
              "Abca3","Muc1","Sftpa1","Sftpb","Sftpd","Scd1", #AT2
              "Scgb1a1","Cyp2f2","Scgb3a2", "Scgb3a1","Lypd2",#Club
              "Muc5ac","Muc5b", # Goblet
              "Tubb4a","Foxa3","Foxj1","Rfx2","Rfx3","Trp73", #Ciliated
              'Krt5', 'Krt17','Krt15','Trp63','Id1','Icam1','Epas1','Aqp3','Sfn','Perp','Fxyd3','Sdc1','Gstm2','F3','Abi3bp','Adh7', # Basal
              'Bex2','Ascl1','Meis2','Hes6','Hoxb5','Foxa2','Sox4','Rora','Isl1','Id4', 'Neurod1','Neurod4','Nhlh1','Nhlh2',#NE/neuronal
              'Pou2f3','Trpm5','Ascl2','Ehf',
              'Lrmp','Gng13','Ltc4s','Alox5ap','Avil','Alox5','Atp2a3','Plk2', #tuft
              "Cftr","Ascl3", 'Stap1','Atp6v1c2','Pparg','Rasd1','Slc12a2', #ionocyte
              "Gja1","Nkx2-1","Epcam", # Lung lineage
              'Yap1','Wwtr1','Sox2','Cd44','Hes1', # Stem-like
             "Venus","fLuc", "Top2a","Mki67",
            'GFP.CDS','CellTag.UTR'] # Tumor markers

sc.set_figure_params(scanpy=True, fontsize=20) 
sc.pl.dotplot(
    adata_2,figsize=[28,10],
    var_names=more_types,
    groupby='leiden_scVI_1.2',
    use_raw=False,
    layer="norm",show=False,
    color_map="cmo.dense", var_group_rotation=35,standard_scale='var',
    save=False)

In [ ]:
adata_2.write_h5ad("050225_RPM_Orgs_Allo_AllUpdated_adata2.h5ad")

In [ ]:
adata_2=sc.read_h5ad("050225_RPM_Orgs_Allo_AllUpdated_adata2.h5ad")

In [ ]:
#feature plots
more_types=["Ascl1", "Neurod1","Atoh1","Pou2f3","Trp63","fLuc", "GFP.CDS","CellTag.UTR"] #Endothelial

sc.pl.umap(
    adata_2,
    color=more_types,
    use_raw=False,
    legend_loc= "on data",
    color_map="cmo.dense",
    ncols=4,s=30,
    frameon=False,
    vmax="p99.5",
    layer="norm",
    save=False
)

In [ ]:
# Find cluster markers for each leiden cluster to aid filtering
sc.tl.rank_genes_groups(adata_2, 'leiden_scVI_1.2', method='wilcoxon', layer='norm', use_raw=False)
pd.DataFrame(adata.uns['rank_genes_groups']['names']).head(50)

In [ ]:
# Extract the ranked genes (names) for all clusters
marker_genes = pd.DataFrame(adata_2.uns['rank_genes_groups']['names'])

# Save to CSV
marker_genes.to_csv('050225_RPM_Orgs_Allo_adata2_DEGsbyleiden_final.csv', index=False)

In [ ]:
# Remove Cluster 12 as potential doublets
bad_clust=['12']

#Filter out bad clusters
to_keep=(~adata_2.obs['leiden_scVI_1.2'].isin(bad_clust))

#Copy over to new anndata object
adata_3 = adata_2[to_keep].copy()

## ITERATION 3 (Final iteration for Fig. 2d, (Final Fig. 1i))

In [ ]:
#HVG via Scanpy
#Note here that if you run with a batch_key with few cells, will get b'reciprocal condition number error
sc.pp.highly_variable_genes(
    adata_3,n_top_genes=5000,
    subset=False,
    layer="counts",
    flavor="seurat_v3"
)

In [ ]:
adata_3.var['mean_'] = np.array(adata_3.X.mean(0))[0]
adata_3.var['frac_zero'] = 1 - np.array((adata_3.X > 0).sum(0))[0] / adata_3.shape[0]

fig, ax = plt.subplots(figsize=(9,6))
ax.scatter(adata_3.var.mean_, adata_3.var.frac_zero, s=1)
ax.set_xscale("log")

In [ ]:
#Calculate Poisson gene selection
df_poisson = scvi.data.poisson_gene_selection(
    adata_3, n_top_genes=5000, batch_key="Batch", inplace=False
)

df_poisson[df_poisson.highly_variable].sort_values('prob_zero_enrichment_rank')

pd.crosstab(df_poisson.highly_variable, adata_3.var.highly_variable)

is_hvg = df_poisson.highly_variable

adata_3.varm['df_poisson']= df_poisson

adata_query = adata_3[:, is_hvg].copy()
print(adata_query)

In [ ]:
scvi.model.SCVI.setup_anndata(
    adata_query,
    layer="counts",
    batch_key='Batch',
    continuous_covariate_keys=["pct_counts_mito"]
)

model = scvi.model.SCVI(adata_query)

In [ ]:
#Train and run scvi

#Training parameters
train_kwargs = dict(
    early_stopping=True,
    early_stopping_patience=20,
    enable_model_summary=True,
    enable_progress_bar=True,
    enable_checkpointing=True,
    max_epochs=500
)

#Train and run model
#Be sure GPU is enabled to run this
model.train(**train_kwargs)

In [ ]:
#Fit model to data
#Get latent representation of model to apply to UMAP
latent = model.get_latent_representation()

adata_3.obsm["X_scVI_1.3"] = latent

In [ ]:
#Calculate neighbors using scVI model input
sc.pp.neighbors(adata_3, use_rep="X_scVI_1.3")
sc.tl.umap(adata_3, min_dist=1)

#Run leiden clustering based on neighbors
sc.tl.leiden(adata_3, key_added="leiden_scVI_1.3", resolution=0.5)

### Visualize clusters, sample distribution, and QC metrics

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))
sc.pl.umap(adata_3, color="leiden_scVI_1.3", legend_loc="on data", legend_fontsize='large',ax=ax, s=30, frameon=False, save=False)
fig, ax = plt.subplots(figsize=(8, 6))
sc.pl.umap(adata_3, color="UnID", legend_loc="right margin", ax=ax, s=30, frameon=False, save=False)
fig, ax = plt.subplots(figsize=(8, 6))
sc.pl.umap(adata_3, color="GenoCT", legend_loc="right margin", ax=ax, s=30, frameon=False, save=False)


#Additional QC bar graphs
adata_3.obs['cluster'] = adata_3.obs["leiden_scVI_1.3"].copy()

#Plot Log1p total counts
fig, ax = plt.subplots(figsize=(20,6))
sns.boxenplot(data=adata_3.obs, x="cluster", y="log1p_total_counts", ax=ax)

#Plot Pct counts mito
fig, ax = plt.subplots(figsize=(20,6))
sns.boxenplot(data=adata_3.obs, x="cluster", y="pct_counts_mito", ax=ax)

In [ ]:
# Dot plot key cell type markers
more_types=["Col14a1", "Acta2","Myh11","Tagln","Mustn1", #fibroblast
              "Lpl","Lipa","Pparg","Plin2","Ear1","Spp1", #lipofibroblast/osteoblastic
              "Ptprc","Mertk","Mrc1","Ly75","Adgre1","Itgax","Cd68","Csf1r","Mafb","Msr1","Arg1","Adgre4", #Macs/Myeloid
              "Cx3cr1","Itgam","Cd14", #Monocytes
              "S100a9","S100a8","Mmp9","Csf3r","Cxcr2","Ly6g", #Neuts
              "Batf3","Xcr1","Clec9a","Ccl17","Ccl22", #DC
              "Cd3d","Cd3e","Cd3g","Cd28","Cd8a","Cd4","Foxp3", # Tcell
              "Gzma","Ncr1","Gzmb", #NK
              "Fcmr","Cd19","Fcer2a","Pax5","Cd22","Cd79b","Cd79a", #B cells
              "Slamf7", "Prdm1", #Plasma
              "Mcam","Pecam1","Icam2","Cd36","Cd93", #Endothelial
    "Pdpn","Cav1","Cav2","Hopx","Timp3","Sema3f","Serpine1", #AT1
              "Abca3","Muc1","Sftpa1","Sftpb","Sftpd","Scd1", #AT2
              "Scgb1a1","Cyp2f2","Scgb3a2", "Scgb3a1","Lypd2",#Club
              "Muc5ac","Muc5b", # Goblet
              "Tubb4a","Foxa3","Foxj1","Rfx2","Rfx3","Trp73", #Ciliated
              'Krt5', 'Krt17','Krt15','Trp63','Id1','Icam1','Epas1','Aqp3','Sfn','Perp','Fxyd3','Sdc1','Gstm2','F3','Abi3bp','Adh7', # Basal
              'Bex2','Ascl1','Meis2','Hes6','Hoxb5','Foxa2','Sox4','Rora','Isl1','Id4', 'Neurod1','Neurod4','Nhlh1','Nhlh2',#NE/neuronal
              'Pou2f3','Trpm5','Ascl2','Ehf',
              'Lrmp','Gng13','Ltc4s','Alox5ap','Avil','Alox5','Atp2a3','Plk2', #tuft
              "Cftr","Ascl3", 'Stap1','Atp6v1c2','Pparg','Rasd1','Slc12a2', #ionocyte
              "Gja1","Nkx2-1","Epcam", # Lung lineage
              'Yap1','Wwtr1','Sox2','Cd44','Hes1', # Stem-like
             "Venus","fLuc", "Top2a","Mki67",
            'GFP.CDS','CellTag.UTR'] # Tumor markers

sc.set_figure_params(scanpy=True, fontsize=20) 
sc.pl.dotplot(
    adata_3,figsize=[28,5],
    var_names=more_types,
    groupby='leiden_scVI_1.3',
    use_raw=False,
    layer="norm",show=False,
    color_map="cmo.dense", var_group_rotation=35,standard_scale='var',
    save=False)

In [ ]:
#feature plots
more_types=["Ascl1", "Neurod1","Atoh1","Pou2f3","Trp63","fLuc", "GFP.CDS","CellTag.UTR"] #Endothelial

sc.pl.umap(
    adata_3,
    color=more_types,
    use_raw=False,
    legend_loc= "on data",
    color_map="cmo.dense",
    ncols=4,s=30,
    frameon=False,
    vmax="p99.5",
    layer="norm",
    save=False
)

In [ ]:
# Find cluster markers for each leiden cluster to confirm tumour clusters
sc.tl.rank_genes_groups(adata_3, 'leiden_scVI_1.3', method='wilcoxon', layer='norm', use_raw=False)
pd.DataFrame(adata_3.uns['rank_genes_groups']['names']).head(50)

In [ ]:
# Extract the ranked genes (names) for all clusters
marker_genes = pd.DataFrame(adata_3.uns['rank_genes_groups']['names'])

# Save to CSV
marker_genes.to_csv('050225_adata3_RPM_Orgs_Allo_OGandAllo3_leiden_cluster_markers.csv', index=False)

In [ ]:
adata_3.write_h5ad("050325_RPM_Orgs_Allos_Updated_adata3.h5ad")

## End of analysis in Scanpy, move to Seurat in R for calculation of gene signatures and additional plot generation/visualization. To do this, convert resulting h5ad anndata object from this script as a Seurat object in R.